In [1]:
import os,sys
sys.path.insert(1, os.path.join(os.getcwd()  , '..'))

In [2]:
import shallowsim as sb
import pandas as pd
import math

In [3]:
args = sb.ModelArgs()
gpu_blackwell = sb.get_gpu_info('./device/gpu_info.csv', device_list=['DGX-B200','GB200-NVL72','H200','H800','H20-3E'] ,print_console=True ) 

| gpu_type    |   sm |   comm_sm |   fp16 |   fp8 |   fp4 |   mem |   mem_bw |   nvlink_bw |   pcie_bw |   gpu_per_node |
|:------------|-----:|----------:|-------:|------:|------:|------:|---------:|------------:|----------:|---------------:|
| DGX-B300    |  160 |        20 |   3375 |  7500 | 15000 |   288 |     9600 |         900 |       100 |              8 |
| DGX-B200    |  160 |        20 |   2250 |  4500 |  9000 |   180 |     8000 |         900 |       100 |              8 |
| GB200-NVL72 |  160 |        20 |   2500 |  5000 | 10000 |   192 |     8000 |         900 |       100 |             72 |
| GB300-NVL72 |  160 |        20 |   3750 |  7500 | 15000 |   288 |     9600 |         900 |       100 |             72 |
| H200        |  132 |        20 |    989 |  1979 |     0 |   141 |     4800 |         450 |        50 |              8 |
| H800        |  132 |        20 |    989 |  1979 |     0 |    80 |     4000 |         200 |        50 |              8 |
| H20         |   78 |  

In [4]:
seq_len = 4383
kv_cache_rate = 0.563
decode_len = 1210
bs_list =[ 16, 32, 64, 128, 256, 512]
eplist = [ 8 , 16, 36, 72, 144, 320]

In [5]:
detail,summary = sb.prefill_time(args,gpu_blackwell,seq_len, kv_cache_rate, tp=4, dp=8)

In [6]:
detail

GPU,Layers,DGX-B200,GB200-NVL72,H200,H800,H20-3E
MLA,3.0,2.310493,2.079444,5.848370,5.857117,37.797622
DenseMLP,3.0,0.574656,0.522749,2.527005,2.545534,15.933123
TP_MLA,58.0,0.720955,0.663193,1.683756,1.881772,9.671069
Shared Expert,58.0,0.020595,0.019153,0.077915,0.079974,0.450307
Combine,58.0,0.578740,0.204216,1.107479,1.107479,1.107479
Overlap1,58.0,-0.162811,-0.478130,-0.654192,-0.854267,-9.013897
Routed Expert,58.0,0.164760,0.153225,0.623321,0.639792,3.602459
Dispatch,58.0,0.182185,0.088554,0.314370,0.314370,0.314370
Overlap2,58.0,0.017425,-0.064671,-0.308951,-0.325422,-3.288089


In [7]:
summary

GPU,DGX-B200,GB200-NVL72,H200,H800,H20-3E
Compute,61.221441,56.269710,163.455698,176.097174,957.174671
Comm,44.133625,16.980641,82.467251,82.467251,82.467251
Sum,62.232083,56.269710,163.455698,176.097174,957.174671


In [8]:
tp=4
_ , ttft_sum = sb.prefill_time(args,gpu_blackwell,seq_len, kv_cache_rate, tp=tp, dp=8, print_console=False)
print(ttft_sum.apply(lambda x: seq_len/tp * (1000/ x)).loc['Sum'].to_markdown(floatfmt=".1f"))


| GPU         |     Sum |
|:------------|--------:|
| DGX-B200    | 17607.5 |
| GB200-NVL72 | 19473.2 |
| H200        |  6703.7 |
| H800        |  6222.4 |
| H20-3E      |  1144.8 |
